In [ ]:
from PIL import Image
import numpy as np
import os
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from keras.models import  * 
from keras.layers import  *
from keras import backend as K
from keras.callbacks import  *
from keras.optimizers import SGD
import tensorflow as tf

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

dict1={}
dict1.update(unpickle("C:\\Users\\dell\\Desktop\\Datasets\\cifar-10-batches-py\\data_batch_1"))
dict1.update(unpickle("C:\\Users\\dell\\Desktop\\Datasets\\cifar-10-batches-py\\data_batch_2"))
dict1.update(unpickle("C:\\Users\\dell\\Desktop\\Datasets\\cifar-10-batches-py\\data_batch_3"))
dict1.update(unpickle("C:\\Users\\dell\\Desktop\\Datasets\\cifar-10-batches-py\\data_batch_4"))
dict1.update(unpickle("C:\\Users\\dell\\Desktop\\Datasets\\cifar-10-batches-py\\data_batch_5"))
#print(dict1[b'data'])
#print(dict1[b'labels'])
X=dict1[b'data']                # shape of X is (10000,3072)
y=np.asarray(dict1[b'labels'])  # shape of y is (10000)


f=32
X=X.copy()
X_red=X[:,0:1024]
X_green=X[:,1024:2048]
X_blue=X[:,2048:3072]
# print(X_red.shape)
clf=PCA(n_components=16*16)
X_red=clf.fit_transform(X_red)
# print(X_red.shape)
X_green=clf.fit_transform(X_green)
X_blue=clf.fit_transform(X_blue)

clf=RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_red,y)
X_red=X_red[:,clf.feature_importances_.argsort()[::-1][:12*12]]
clf.fit(X_green,y)
X_green=X_green[:,clf.feature_importances_.argsort()[::-1][:12*12]]
clf.fit(X_blue,y)
X_blue=X_blue[:,clf.feature_importances_.argsort()[::-1][:12*12]]
X_red=np.reshape(X_red,(X_red.shape[0],12,12))         #10000 red images has been reshaped into 12*12
X_green=np.reshape(X_green,(X_green.shape[0],12,12))   #10000 green images has been reshaped into 12*12
X_blue=np.reshape(X_blue,(X_blue.shape[0],12,12))      #10000 blue images has been reshaped into 12*12

img=np.zeros((10000,12,12,3))
img[:,:,:,0]=X_red
img[:,:,:,1]=X_green
img[:,:,:,2]=X_blue
img=img.reshape((-1, 12, 12, 3))
img.shape

inputs=Input(shape=(12,12,3))
x=Conv2D(16,(3,3), activation='relu',padding='same')(inputs)
x=Conv2D(16,(3,3), activation='relu',padding='same')(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=BatchNormalization()(x)
x=Conv2D(16,(3,3), activation='relu',padding='same')(x)
x=Conv2D(16,(3,3), activation='relu',padding='same')(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=BatchNormalization()(x)
x=Conv2D(16,(3,3), activation='relu',padding='same')(x)
x=Conv2D(16,(3,3), activation='relu',padding='same')(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=BatchNormalization()(x)
x=Dropout(0.25)(x)
x=Flatten()(x)
x=Dense(10)(x)                                         #10 for number of classes 
x=Activation('softmax')(x)
output=Activation('softmax')(x)
model=Model([inputs], output)
sgd=SGD(lr=0.01,decay=1e-6, momentum=0.9, nesterov=True)
#print(sgd)
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(img,y,nb_epoch=1000)


In [ ]:
import SwarmPackagePy
from SwarmPackagePy import testFunctions as tf
from SwarmPackagePy import animation, animation3D

alh=SwarmPackagePy.pso(50,tf.easom_function,-10, 10, 2, 20,w=0.5, c1=1, c2=1)
# print(alh)
animation(alh.get_agents(), tf.easom_function, -10, 10)
animation3D(alh.get_agents(), tf.easom_function, -10, 10)

# # model.compile(optimizer=alh,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# # model.fit(img,y,nb_epoch=50)